In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
import joblib


In [28]:
data_movies = pd.read_csv('movie_dataset.csv')
data =data_movies.copy()

In [29]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [30]:
data.shape

(4803, 24)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [32]:
data.describe()


,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [33]:
dict_nan = {"columns": [],"NaN":[]}
for col , nans in zip(data.columns , data.isna().sum()):
    dict_nan["columns"].append(col)
    dict_nan["NaN"].append(nans)
nan_df =pd.DataFrame(dict_nan)
nan_df

,columns,NaN
0,index,0
1,budget,0
2,genres,28
3,homepage,3091
4,id,0
5,keywords,412
6,original_language,0
7,original_title,0
8,overview,3
9,popularity,0


In [34]:
#columns_with_missing = nan_df[nan_df['NaN'] > 45]['columns'].tolist()
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [35]:
data.fillna('')

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""En

#selecting features

In [36]:

selected_features = ['genres', 'keywords',
       'original_language', 'original_title', 'popularity',
       'production_companies', 'production_countries', 'tagline', 'title','cast', 'crew', 'director']
print(selected_features)

['genres', 'keywords', 'original_language', 'original_title', 'popularity', 'production_companies', 'production_countries', 'tagline', 'title', 'cast', 'crew', 'director']


In [37]:
for feature in selected_features:
  data[feature] = data[feature].fillna('')

In [38]:
combined_features = data['genres']+' '+data['keywords']+' '+data['tagline']+' '+data['cast']+' '+data['director']+' '+data['original_language']+' '+data['original_title']+' '+data['production_countries']+' '+data['title']

In [39]:
#columns_with_missing = nan_df[nan_df['NaN'] > 45]['columns'].tolist()
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [40]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

#vectorizing data

In [41]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)


In [42]:
print(feature_vectors)

  (0, 246)	0.0677900284307754
  (0, 326)	0.07775680162562094
  (0, 5992)	0.09587577213878827
  (0, 15284)	0.0894508079718828
  (0, 6170)	0.08934660559247938
  (0, 4172)	0.18463159885123379
  (0, 3468)	0.1916760447566187
  (0, 6643)	0.141120571519122
  (0, 16195)	0.2892578909099733
  (0, 18720)	0.10720832116947819
  (0, 3649)	0.21542614929329296
  (0, 16072)	0.18167830063357066
  (0, 5619)	0.19599695117154073
  (0, 17209)	0.047670449911719225
  (0, 19172)	0.1082095135898878
  (0, 12587)	0.06411638891228001
  (0, 12941)	0.22679150161359993
  (0, 15020)	0.12942737443485688
  (0, 19182)	0.204060796972986
  (0, 19489)	0.1732662465309268
  (0, 14988)	0.18793310537009547
  (0, 15821)	0.17776071639685165
  (0, 18812)	0.17126322263106375
  (0, 16464)	0.13076240806972966
  (0, 9886)	0.19599695117154073
  :	:
  (4802, 8706)	0.02608345463471008
  (4802, 18290)	0.02998382798416884
  (4802, 12113)	0.026078022085294195
  (4802, 18216)	0.02843379830576389
  (4802, 16405)	0.030034634201838238
  (4802, 

#calculating the cosine similarity

In [43]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06921117 0.05952367 ... 0.00984134 0.01341664 0.01048617]
 [0.06921117 1.         0.03181974 ... 0.04738642 0.00919524 0.0081277 ]
 [0.05952367 0.03181974 1.         ... 0.0091033  0.04610114 0.00969977]
 ...
 [0.00984134 0.04738642 0.0091033  ... 1.         0.00724168 0.0654543 ]
 [0.01341664 0.00919524 0.04610114 ... 0.00724168 1.         0.00771617]
 [0.01048617 0.0081277  0.00969977 ... 0.0654543  0.00771617 1.        ]]


In [44]:
print(similarity.shape)

(4803, 4803)


#inputing the name movie


In [45]:
movie_name = input(' Enter your favourite movie name : ')
print(movie_name)

hitman


In [46]:
list_of_all_titles = data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [47]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)
close_match = find_close_match[0]
print(close_match)

['Hitman', 'Batman', 'Batman']
Hitman


In [48]:
index_of_the_movie = data[data.title == close_match]['index'].values[0]
print(index_of_the_movie)

2398


In [49]:

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)
len(similarity_score)

[(0, 0.020097268610050734), (1, 0.01407775589249511), (2, 0.04215420782521843), (3, 0.07028866251266118), (4, 0.015277045253245884), (5, 0.03917361932474267), (6, 0.010023159304798046), (7, 0.023282864446429383), (8, 0.01326688327453719), (9, 0.013745050857490166), (10, 0.01494518844631492), (11, 0.12834835724486643), (12, 0.020126480603368354), (13, 0.012271501604554546), (14, 0.02075208441686364), (15, 0.01625823666896477), (16, 0.024737866250987887), (17, 0.013368833240566511), (18, 0.01441403783443826), (19, 0.015567467662677891), (20, 0.027375896289074195), (21, 0.026649548629310373), (22, 0.011300801752774172), (23, 0.010399000644394078), (24, 0.023695917140635903), (25, 0.015741137453159568), (26, 0.024998412227984823), (27, 0.01838519082252689), (28, 0.01913733288434365), (29, 0.04030703167996747), (30, 0.04145544453511528), (31, 0.028453306012639203), (32, 0.007961190120315056), (33, 0.0247478765809628), (34, 0.009797290586751538), (35, 0.015003491406498561), (36, 0.0125942541

4803

#sorting movies based on their similarity


In [50]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(2398, 1.0), (1278, 0.1738913271986337), (731, 0.16667386224112907), (963, 0.14066097640436037), (694, 0.13910644168165223), (821, 0.13463854376918333), (11, 0.12834835724486643), (1096, 0.1263163266536471), (1114, 0.12556368344966426), (975, 0.1255172255797234), (2645, 0.12333887612298736), (2583, 0.12275990125041515), (248, 0.12089685631527991), (2853, 0.11904500706986684), (2641, 0.11800610490629754), (756, 0.11435628245368473), (213, 0.11315815229356382), (1020, 0.11256549583844222), (750, 0.1112449943443337), (1139, 0.11002951552031441), (512, 0.10856580965534877), (3040, 0.10474284251258827), (84, 0.10469680370192733), (2412, 0.10366508992601106), (4475, 0.10343895816106013), (228, 0.09972946002445846), (1245, 0.09921224907900418), (3956, 0.09765630732643149), (2508, 0.09747749302792925), (2711, 0.09680096947128793), (433, 0.09649786834653806), (1187, 0.0963778911523556), (1013, 0.0960778573340281), (767, 0.09500206546702097), (295, 0.09402404051064953), (939, 0.0927062222210705

#displaying movies


In [51]:
print('Movies suggested for you : \n')
i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index =data[data.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Hitman
2 . The Gunman
3 . The Jackal
4 . Ninja Assassin
5 . The Bourne Identity


In [52]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming you have your data and combined_features prepared
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(combined_features)

# Save the vectorizer to a file
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
data.to_csv('movie_dataset1.csv', index=False)